Steps:
- Enable data augmentation, and precompute=True
- Use lr_find() to finmd highest learning rate where loss is clearly improving
- Train last layer from pre-computed activations for 1-2 epochs
- Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
- Unfreeze all layers
- Set earlier layers to 3x-10x lower learning rate than next higher layer
- Use lr_find() again
- Train full network with cycle=mult=2 until over-fitting

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from sklearn import metrics
from fastai.conv_learner import *

In [ ]:
PATH = 'data/dogcatbreed/'

In [ ]:
bs=20
f_model = resnext101_64

In [ ]:
# Data preparation steps if you are using Crestle:

os.makedirs('data/dogcatbreed/models', exist_ok=True)
os.makedirs('/cache/dogcatbreed/tmp', exist_ok=True)

In [ ]:
label_csv= f'{PATH}train.csv'
n = len(list(open(label_csv))) - 1 #count row in file, exclude the header row
val_idxs = get_cv_idxs(n) # function choses random 20% data for valdation set

In [ ]:
def get_data(sz, bs):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test', bs=bs)

In [ ]:
sz=224
data = get_data(sz,bs)

In [ ]:
data = data.resize(int(sz*1.3), 'tmp')

In [ ]:
learn = ConvLearner.pretrained(f_model, data, ps=0.5) #ps sets the dropout rate

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 1e-2

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
lrs = np.array([lr/9,lr/3,lr])

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

In [ ]:
learn.save(f'r101{sz}')

In [ ]:
learn.sched.plot_loss()

In [ ]:
sz=299

In [ ]:
learn.set_data(get_data(sz,bs))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

In [ ]:
learn.save(f'r101{sz}')

# Run on test data now and create a submission

In [ ]:
log_preds,y = learn.TTA(is_test=True) #use test dataset not validation dataset
probs = np.mean(np.exp(log_preds),0)

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])
df.head()

In [ ]:
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}fs2-subm1.gz', compression='gzip', index=False)
FileLink(f'{SUBM}fs2-subm1.gz')